<a href="https://colab.research.google.com/github/rakshitrk/handwriten_digit_recogination/blob/master/validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torchvision as tv
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler

In [0]:
b = 10
n_epoch = 50
val_split = 0.2

In [0]:
train_data = tv.datasets.MNIST(root='./data',train=True,transform=transforms.ToTensor(),download=True)
test_data = tv.datasets.MNIST(root='./data',train=False,transform=transforms.ToTensor(),download=True)



In [0]:
#spliting test data to test and validation data
size = len(train_data)
indices = list(range(size))
split = int(np.floor(val_split * size))
np.random.shuffle(indices)
train_indices,val_indices = indices[split:],indices[:split]

train = SubsetRandomSampler(train_indices)
val = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(train_data,batch_size=b,sampler=train)
val_loader = torch.utils.data.DataLoader(train_data,batch_size=b,sampler=val)
test_loader = torch.utils.data.DataLoader(test_data,batch_size=b)



In [45]:
#initalising model
class Model(nn.Module):
  def __init__(self):
    super(Model,self).__init__()
    h1 = 512
    h2 = 512
    self.l1 = nn.Linear(28*28,h1)
    self.l2 = nn.Linear(h1,h2)
    self.l3 = nn.Linear(h2,10)
    self.drop = nn.Dropout(.2)
    
  def forward(self,x):
    
    x = x.view(-1,28*28)
    x = F.relu(self.l1(x))
    x = self.drop(x)
    x = F.relu(self.l2(x))
    x = self.drop(x)
    x = self.l3(x)
    return x
    
  
m = Model()
print(m)

Model(
  (l1): Linear(in_features=784, out_features=512, bias=True)
  (l2): Linear(in_features=512, out_features=512, bias=True)
  (l3): Linear(in_features=512, out_features=10, bias=True)
  (drop): Dropout(p=0.2)
)


In [0]:
critersion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(m.parameters(),lr=0.01)

In [47]:
#training model
val_loss_min=np.Inf
for epoch in range(n_epoch):
  train_loss = 0.0
  val_loss = 0.0
  
  m.train()
  for data,target in train_loader:
    optimizer.zero_grad()
    output = m(data)
    loss = critersion(output,target)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()*data.size(0)
    
  m.eval()
  for data,target in train_loader:
      optimizer.zero_grad()
      output = m(data)
      loss = critersion(output,target)
      val_loss += loss.item()*data.size(0)
      
  train_loss = train_loss/len(train_loader.dataset)
  val_loss = val_loss/len(val_loader.dataset)
  
  print('Epoch:{} \ttraining loss:{:.6f} \tvalidation loss:{:.6f}'
        .format(epoch+1,train_loss,val_loss))
  
  
  if val_loss <= val_loss_min:
    print('Validation loss dec:{:.6f}-->{:.6f}\t.....SAVING.........'
          .format(val_loss_min,val_loss))
    
    torch.save(m.state_dict(),'m.pt')
    val_loss_min = val_loss
    
  
      
    
    
  

Epoch:1 	training loss:0.528340 	validation loss:0.232492
Validation loss dec:inf-->0.232492	.....SAVING.........
Epoch:2 	training loss:0.210947 	validation loss:0.153556
Validation loss dec:0.232492-->0.153556	.....SAVING.........
Epoch:3 	training loss:0.152601 	validation loss:0.112726
Validation loss dec:0.153556-->0.112726	.....SAVING.........
Epoch:4 	training loss:0.119117 	validation loss:0.087895
Validation loss dec:0.112726-->0.087895	.....SAVING.........
Epoch:5 	training loss:0.097320 	validation loss:0.070449
Validation loss dec:0.087895-->0.070449	.....SAVING.........
Epoch:6 	training loss:0.083265 	validation loss:0.060774
Validation loss dec:0.070449-->0.060774	.....SAVING.........
Epoch:7 	training loss:0.071515 	validation loss:0.051969
Validation loss dec:0.060774-->0.051969	.....SAVING.........
Epoch:8 	training loss:0.062331 	validation loss:0.042162
Validation loss dec:0.051969-->0.042162	.....SAVING.........
Epoch:9 	training loss:0.054372 	validation loss:0.03

In [0]:
m.load_state_dict(torch.load('m.pt'))

In [0]:
for data,target in test_loader:
  output = m(data)
  _, pred = torch.max(output,1)

In [63]:
correct = 0
total = 0

for data in test_loader:
  images,labels = data
  output = m(images)
  _, pred = torch.max(output,1)
  
  total += labels.size(0)
  correct += (pred == labels).sum().item()
  
  acc = 100*correct/total
print('Accuracy of model:{}%'.format(acc))
  

Accuracy of model:98.28%
